# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook and serves as an extension of the API documentation to show what the results will look like depending on what inputs are provided.

<div class="alert alert-block alert-warning">
<b>Note:</b>
As of v0.5, most outputs, including availability, return values as a pandas DataFrame, regardless of the aggregation level.
</div>


In [1]:
from pprint import pprint

import pandas as pd

from wombat.core import Simulation, Metrics

# Clean up the aesthetics for the pandas outputs
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 10)

## Table of Contents

Below is a list of top-level sections to demonstrate how to use WOMBAT's `Metrics` class methods and an explanation of each individual metric.
 - [Setup](#setup): Running a simulation to gather the results
 - [Availability](#availability): Time-based and energy-based availability
 - [Capacity Factor](#capacity-factor): Gross and net capacity factor
 - [Task Completion Rate](#task-completion-rate): Task completion metrics
 - [Equipment Costs](#equipment-costs): Cost breakdowns by servicing equipment
 - [Service Equipment Utilization Rate](#service-equipment-utilization-rate): Utilization of servicing equipment
 - [Vessel-Crew Hours at Sea](#vessel-crew-hours-at-sea): Number of crew or vessel hours spent at sea
 - [Number of Tows](#number-of-tows): Number of tows breakdowns
 - [Labor Costs](#labor-costs): Breakdown of labor costs
 - [Equipment and Labor Costs](#equipment-and-labor-costs): Combined servicing equipment and labor cost breakdown
 - [Component Costs](#component-costs): Materials costs
 - [Fixed Cost Impacts](#fixed-cost-impacts): Total fixed costs
 - [OpEx](#opex): Project OpEx
 - [Process Times](#process-times): Timing of various stages of repair and maintenance
 - [Power Production](#power-production): Potential and actually produced power
 - [Net Present Value](#net-present-value): Project NPV calculator
 - [PySAM-Powered Results](#pysam-powered-results): PySAM results, if configuration is provided

<a id="setup"></a>
## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Delete the log files now that they're loaded in
sim.env.cleanup_log_files(log_only=False)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

TypeError: cleanup_log_files() got an unexpected keyword argument 'log_only'

<a id="availability"></a>
## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm").values[0][0]
print(f"Project total: {total * 100:.1f}%")

Project total: 94.2%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,...,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.97,0.97,0.95,0.97,0.87,...,0.97,0.97,0.97,0.97,0.91


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.97
2004,0.96
2005,0.95
2006,0.94
2007,0.93
2008,0.92
2009,0.93
2010,0.95
2011,0.94


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.94
2,0.94
3,0.93
4,0.93
5,0.93
6,0.94
7,0.94
8,0.94
9,0.95


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm")

windfarm
year month          
2003 1          0.97
     2          0.96
     3          0.96
     4          0.96
     5          0.94
...              ...
2012 8          0.92
     9          0.95
     10         0.97
     11         0.97
     12         0.96

[120 rows x 1 columns]

<a id="capacity-factor"></a>
## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm").values[0][0]
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm").values[0][0]
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.45%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,...,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.46,0.46,0.45,0.46,0.41,...,0.46,0.47,0.46,0.46,0.44


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.45
2005,0.46
2006,0.44
2007,0.47
2008,0.48
2009,0.44
2010,0.40
2011,0.47


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.59
2,0.49
3,0.49
4,0.40
5,0.37
6,0.31
7,0.32
8,0.35
9,0.45


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm")

windfarm
year month          
2003 1          0.61
     2          0.35
     3          0.37
     4          0.52
     5          0.33
...              ...
2012 8          0.29
     9          0.54
     10         0.52
     11         0.53
     12         0.60

[120 rows x 1 columns]

<a id="task-completion-rate"></a>
## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project").values[0][0]
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project").values[0][0]
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project").values[0][0]
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 99%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,1.00
2004,0.97
2005,1.02
2006,1.00
2007,1.00
2008,1.00
2009,1.00
2010,1.00
2011,0.99


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,0.74
2,0.86
3,0.92
4,1.08
5,1.17
6,1.08
7,1.12
8,1.07
9,1.09


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year")

Completion Rate
year month                 
2003 1                 0.81
     2                 1.16
     3                 0.94
     4                 1.05
     5                 1.00
...                     ...
2012 8                 1.12
     9                 1.10
     10                0.90
     11                1.13
     12                1.07

[120 rows x 1 columns]

<a id="equipment-costs"></a>
## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $362,075.87/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,103,270.83","6,098,841.15","6,105,330.73","1,827,207.02","66,763,559.03"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"6,189,086.81"
2004,"6,602,739.79"
2005,"7,226,439.80"
2006,"11,007,437.50"
2007,"9,372,820.64"
2008,"10,436,317.71"
2009,"8,012,885.09"
2010,"6,138,552.97"
2011,"10,498,092.53"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"500,664.06","501,174.48","498,111.98","77,681.96","5,792,187.50"
2,"463,877.60","461,161.46","462,164.06","116,561.70","5,959,647.57"
3,"509,578.12","510,872.40","509,760.42","435,812.50","7,836,336.81"
4,"510,489.58","511,820.31","511,747.40","18,067.04","7,032,243.06"
5,"529,684.90","527,989.58","528,463.54","247,395.83","9,035,711.81"
6,"510,033.85","509,578.12","510,179.69","154,655.71","3,108,494.79"
7,"531,380.21","530,486.98","531,489.58","216,464.43","4,235,994.79"
8,"531,052.08","530,013.02","531,088.54","161,500.00","6,137,730.90"
9,"501,557.29","501,338.54","501,484.38",0.00,"6,253,125.00"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   52,281.25               50,385.42   
     2                   46,356.77               47,250.00   
     3                   52,208.33               53,010.42   
     4                   50,549.48               51,406.25   
     5                   53,447.92               52,645.83   
...                            ...                     ...   
2012 8                   53,885.42               53,885.42   
     9                   48,033.85               48,416.67   
     10                  49,473.96               49,473.96   
     11                  51,333.33               51,333.33   
     12                  51,916.67               50,658.85   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   52,281.25                  0.00               0.00  
     2                   46,356.77                  0.00               0.00  
     3                   53,010.42                  0.00               0.00  
     4                   51,406.25                  0.00               0.00  
     5                   52,572.92                  0.00               0.00  
...                            ...                   ...                ...  
2012 8                   53,958.33                  0.00       1,629,918.40  
     9                   49,091.15                  0.00       2,985,937.50  
     10                  50,312.50                  0.00         765,625.00  
     11                  51,333.33             20,574.65               0.00  
     12                  51,843.75             91,833.33               0.00  

[120 rows x 5 columns]

<a id="service-equipment-utilization-rate"></a>
## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1.00,1.00,1.00,0.93,0.95


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,1.00,1.00,1.00,0.00,0.96
2004,1.00,1.00,1.00,0.94,0.94
2005,1.00,1.00,1.00,0.94,0.94
2006,1.00,1.00,1.00,0.00,0.95
2007,1.00,1.00,1.00,0.94,0.90
2008,1.00,1.00,1.00,0.00,0.98
2009,1.00,1.00,1.00,1.00,0.95
2010,1.00,1.00,1.00,0.93,0.96
2011,1.00,1.00,1.00,0.95,0.94


<a id="vessel-crew-hours-at-sea"></a>
## Vessel-Crew Hours at Sea

The number of vessel hours or crew hours at sea for offshore wind power plant simulations.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed for each vessel simulated
 - `False`: computed only as a sum of all vessels
 
`vessel_crew_assumption`: A dictionary of vessel names (`ServiceEquipment.settings.name`, but also found at `Metrics.service_equipment_names`) and the number of crew onboard at any given time. The application of this assumption transforms the results from vessel hours at sea to crew hours at sea.

In [24]:
# Project total, not broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="project", by_equipment=False)

,Total Crew Hours at Sea
0,"126,590.57"


In [25]:
# Annual project totals, broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="annual", by_equipment=True)

,year,Total Crew Hours at Sea,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Heavy Lift Vessel,Field Support Vessel
0,2003,"7,135.50","1,852.25","1,683.25","1,797.75","1,802.25",0.00
1,2004,"12,499.82","3,402.75","3,414.75","3,381.25","1,656.07",645.00
2,2005,"12,629.00","3,381.50","3,339.75","3,353.50","1,932.25",622.00
3,2006,"12,912.01","3,141.50","3,115.25","3,086.75","3,568.51",0.00
4,2007,"13,867.64","3,311.50","3,282.75","3,356.50","3,280.89",636.00
5,2008,"13,606.18","3,378.75","3,269.00","3,346.75","3,611.68",0.00
6,2009,"12,932.66","3,344.00","3,309.50","3,252.50","1,889.66","1,137.00"
7,2010,"12,133.85","3,245.75","3,234.00","3,174.00","1,880.10",600.00
8,2011,"14,215.90","3,174.25","3,122.75","3,105.25","3,502.15","1,311.50"
9,2012,"14,658.01","3,514.25","3,406.50","3,443.00","3,632.26",662.00


<a id="number-of-tows"></a>
## Number of Tows

The number of tows metric will only produce results if any towing actually occurred, otherwise a dataframe with a value of 0 is returned, as is the case in this demonstration.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_tug` options:
 - `True`: computed for each tugboat (towing vessel)
 - `False`: computed only as a sum of all tugboats
 
`by_direction` options:
 - `True`: computed for each direction (to port or to site)
 - `False`: computed as a sum for both directions

In [26]:
# Project Total
# NOTE: This example has no towing, so it will return 0
metrics.number_of_tows(frequency="project")

,total_tows
0,0


<a id="labor-costs"></a>
## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [27]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [28]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [29]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [30]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [31]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
...                      ...
2012 8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

[120 rows x 1 columns]

<a id="equipment-and-labor-costs"></a>
## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [32]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,942,315.10"
Repair,"16,470,179.69"
Crew Transfer,"1,048,760.42"
Site Travel,0.00
Mobilization,"7,500,000.00"
Weather Delay,"33,075,938.37"
No Requests,"7,550,340.61"
Not in Shift,"18,310,674.57"


In [33]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,942,315.10","2,942,315.10"
Repair,0.00,0.00,0.00,"16,470,179.69","16,470,179.69"
Crew Transfer,0.00,0.00,0.00,"1,048,760.42","1,048,760.42"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"7,500,000.00","7,500,000.00"
Weather Delay,0.00,0.00,0.00,"33,075,938.37","33,075,938.37"
No Requests,0.00,0.00,0.00,"7,550,340.61","7,550,340.61"
Not in Shift,0.00,0.00,0.00,"18,310,674.57","18,310,674.57"


In [34]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False)

total_cost
year reason                    
2003 Maintenance       4,247.40
     Repair        1,606,906.25
     Crew Transfer    79,916.67
     Site Travel           0.00
     Mobilization    500,000.00
...                         ...
2012 Site Travel           0.00
     Mobilization  1,000,000.00
     Weather Delay 5,226,150.17
     No Requests     378,947.92
     Not in Shift  2,218,657.98

[80 rows x 1 columns]

In [35]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False)

total_cost
month reason                    
1     Maintenance     104,945.31
      Repair        1,102,933.16
      Crew Transfer    72,718.75
      Site Travel           0.00
      Mobilization    500,000.00
...                          ...
12    Site Travel           0.00
      Mobilization    500,000.00
      Weather Delay 3,578,712.67
      No Requests     858,377.60
      Not in Shift  1,098,219.62

[96 rows x 1 columns]

In [36]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          24,208.33
           Crew Transfer    2,533.85
           Site Travel          0.00
           Mobilization         0.00
...                              ...
2012 12    Site Travel          0.00
           Mobilization         0.00
           Weather Delay   25,945.31
           No Requests    107,552.08
           Not in Shift    49,994.79

[960 rows x 1 columns]

<a id="component-costs"></a>
## Component Costs

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [37]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
,"15,223,835.29"
turbine,"106,916,873.47"


In [38]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
          delay                  0.00              0.00    7,723,835.29   
          maintenance            0.00              0.00            0.00   
          repair                 0.00              0.00            0.00   
turbine   delay                  0.00              0.00   51,213,118.26   
          maintenance            0.00              0.00    2,942,315.10   
          repair                 0.00              0.00   16,470,179.69   

                         total_cost  
component action                     
          delay        7,723,835.29  
          maintenance          0.00  
          repair               0.00  
turbine   delay       51,213,118.26  
          maintenance  2,942,315.10  
          repair      16,470,179.69

In [39]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost    total_cost
year component                                                                
2003                      0.00              0.00    1,735,736.98  1,735,736.98
     turbine      2,066,000.00              0.00    4,453,349.83  6,519,349.83
2004                      0.00              0.00    2,499,166.67  2,499,166.67
     turbine      3,375,000.00              0.00    4,103,573.12  7,478,573.12
2005                      0.00              0.00    1,000,186.33  1,000,186.33
     turbine      3,217,500.00              0.00    6,226,253.47  9,443,753.47
2006                      0.00              0.00    1,558,049.48  1,558,049.48
     turbine      4,577,500.00              0.00    9,449,388.02 14,026,888.02
2007                      0.00              0.00    1,553,082.80  1,553,082.80
     turbine      3,336,000.00              0.00    7,819,737.85 11,155,737.85
2008                      0.00              0.00    1,424,776.04  1,424,776.04
     turbine      2,924,000.00              0.00    9,011,541.67 11,935,541.67
2009                      0.00              0.00    1,004,700.19  1,004,700.19
     turbine      3,438,500.00              0.00    7,008,184.90 10,446,684.90
2010                      0.00              0.00    1,194,490.47  1,194,490.47
     turbine      3,562,500.00              0.00    4,944,062.50  8,506,562.50
2011                      0.00              0.00    1,852,498.78  1,852,498.78
     turbine      4,365,000.00              0.00    8,645,593.75 13,010,593.75
2012                      0.00              0.00    1,401,147.56  1,401,147.56
     turbine      4,380,500.00              0.00   10,012,688.37 14,393,188.37

In [40]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
month component                                                     
1                          0.00              0.00    1,120,135.09   
      turbine      2,243,500.00              0.00    6,249,684.90   
2                          0.00              0.00    1,390,111.18   
      turbine      2,451,000.00              0.00    6,073,301.22   
3                          0.00              0.00    1,540,986.98   
      turbine      2,534,000.00              0.00    8,261,373.26   
4                          0.00              0.00    1,108,645.83   
      turbine      3,868,500.00              0.00    7,475,721.56   
5                          0.00              0.00    1,078,393.23   
      turbine      5,636,000.00              0.00    9,790,852.43   
6                          0.00              0.00    1,882,890.09   
      turbine      2,831,000.00              0.00    2,910,052.08   
7                          0.00              0.00      783,136.30   
      turbine      4,205,000.00              0.00    5,262,679.69   
8                          0.00              0.00    1,384,406.25   
      turbine      3,824,000.00              0.00    6,506,978.30   
9                          0.00              0.00      505,403.65   
      turbine      3,120,000.00              0.00    7,252,101.56   
10                         0.00              0.00      929,450.52   
      turbine      1,342,500.00              0.00    1,447,395.83   
11                         0.00              0.00    2,133,604.30   
      turbine      1,080,500.00              0.00    4,381,508.68   
12                         0.00              0.00    1,366,671.88   
      turbine      2,106,500.00              0.00    6,062,723.96   

                   total_cost  
month component                
1                1,120,135.09  
      turbine    8,493,184.90  
2                1,390,111.18  
      turbine    8,524,301.22  
3                1,540,986.98  
      turbine   10,795,373.26  
4                1,108,645.83  
      turbine   11,344,221.56  
5                1,078,393.23  
      turbine   15,426,852.43  
6                1,882,890.09  
      turbine    5,741,052.08  
7                  783,136.30  
      turbine    9,467,679.69  
8                1,384,406.25  
      turbine   10,330,978.30  
9                  505,403.65  
      turbine   10,372,101.56  
10                 929,450.52  
      turbine    2,789,895.83  
11               2,133,604.30  
      turbine    5,462,008.68  
12               1,366,671.88  
      turbine    8,169,223.96

In [41]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1                          0.00              0.00      105,419.27   
           turbine         76,500.00              0.00       49,528.65   
     2                          0.00              0.00       85,221.35   
           turbine         21,000.00              0.00       54,742.19   
     3                          0.00              0.00      105,783.85   
           turbine         62,000.00              0.00       52,445.31   
     4                          0.00              0.00       95,520.83   
           turbine         59,000.00              0.00       57,841.15   
     5                          0.00              0.00      112,200.52   
           turbine         77,500.00              0.00       46,466.15   
     6                          0.00              0.00      620,239.58   
           turbine         53,000.00              0.00    1,284,690.97   
     7                          0.00              0.00      125,088.54   
           turbine      1,393,000.00              0.00    2,626,723.96   
     8                          0.00              0.00      111,854.17   
           turbine         70,500.00              0.00       47,395.83   
     9                          0.00              0.00      103,432.29   
           turbine         61,000.00              0.00       51,151.04   
     10                         0.00              0.00       86,388.02   
           turbine         23,000.00              0.00       60,830.73   
     11                         0.00              0.00      108,755.21   
           turbine        108,500.00              0.00       47,031.25   
     12                         0.00              0.00       75,833.33   
           turbine         61,000.00              0.00       74,502.60   
2004 1                          0.00              0.00        3,335.94   
           turbine        185,500.00              0.00      144,611.98   
     2                          0.00              0.00          875.00   
           turbine        107,500.00              0.00      131,742.19   

                       total_cost  
year month component               
2003 1                 105,419.27  
           turbine     126,028.65  
     2                  85,221.35  
           turbine      75,742.19  
     3                 105,783.85  
           turbine     114,445.31  
     4                  95,520.83  
           turbine     116,841.15  
     5                 112,200.52  
           turbine     123,966.15  
     6                 620,239.58  
           turbine   1,337,690.97  
     7                 125,088.54  
           turbine   4,019,723.96  
     8                 111,854.17  
           turbine     117,895.83  
     9                 103,432.29  
           turbine     112,151.04  
     10                 86,388.02  
           turbine      83,830.73  
     11                108,755.21  
           turbine     155,531.25  
     12                 75,833.33  
           turbine     135,502.60  
2004 1                   3,335.94  
           turbine     330,111.98  
     2                     875.00  
           turbine     239,242.19

<a id="fixed-cost-impacts"></a>
## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [42]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [43]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,013,230.75"


In [44]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,013,230.75"


In [45]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,...,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,"16,013,230.75"


In [46]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


In [47]:
# Project annualized totals at the low level by month
metrics.project_fixed_costs(frequency="annual", resolution="low")

,operations
year,
2003,"1,600,008.00"
2004,"1,604,391.58"
2005,"1,600,008.00"
2006,"1,600,008.00"
2007,"1,600,008.00"
2008,"1,604,391.58"
2009,"1,600,008.00"
2010,"1,600,008.00"
2011,"1,600,008.00"


In [48]:
# Project annualized totals at the low level by year and month
metrics.project_fixed_costs(frequency="month-year", resolution="low")

operations
year month            
2003 1      135,891.09
     2      122,740.34
     3      135,891.09
     4      131,507.51
     5      135,891.09
...                ...
2012 8      135,891.09
     9      131,507.51
     10     135,891.09
     11     131,507.51
     12     135,891.09

[120 rows x 1 columns]

<a id="opex"></a>
## OpEx

Here, we will go through the various input definitions to get the project OpEx

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [49]:
metrics.opex("project")

,OpEx
0,"138,153,939.51"


In [50]:
metrics.opex("annual")

,OpEx
year,
2003,"9,855,094.81"
2004,"11,582,131.37"
2005,"12,043,947.80"
2006,"17,184,945.50"
2007,"14,308,828.64"
2008,"14,964,709.29"
2009,"13,051,393.09"
2010,"11,301,060.97"
2011,"16,463,100.53"


In [51]:
metrics.opex("monthly")

,OpEx
month,
1,"10,972,230.89"
2,"11,154,966.54"
3,"13,695,271.15"
4,"13,767,942.46"
5,"17,864,156.56"
6,"8,939,017.24"
7,"11,609,726.89"
8,"13,074,295.45"
9,"12,192,580.28"


In [52]:
metrics.opex("month-year")

OpEx
year month             
2003 1       367,339.01
     2       283,703.88
     3       356,120.26
     4       343,869.49
     5       372,057.76
...                 ...
2012 8     2,548,038.66
     9     4,136,986.67
     10    1,500,776.51
     11      369,082.15
     12      745,643.69

[120 rows x 1 columns]

In [53]:
port_fees = pd.DataFrame([], columns=["port_fees"], index=metrics.labor_costs(frequency="month-year", by_type=False).index)
port_fees.fillna(0)

port_fees
year month           
2003 1              0
     2              0
     3              0
     4              0
     5              0
...               ...
2012 8              0
     9              0
     10             0
     11             0
     12             0

[120 rows x 1 columns]

<a id="process-times"></a>
## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [54]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime
category,,,
annual service,"1,330,477.79","93,144.00","96,274.50"
major repair,"102,153.46","2,127.64","2,197.50"
major replacement,"218,594.03","9,243.67","218,594.03"
manual reset,"357,194.22","41,288.75","39,415.25"
medium repair,"16,171.85","10,242.00","10,762.75"
minor repair,"109,734.64","34,423.50","36,060.00"


<a id="power-production"></a>
## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [55]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,455,131,104.00"


In [56]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,...,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,455,131,104.00","122,039,755.00","122,208,348.00","119,373,911.50","121,883,146.50",...,"122,117,094.00","122,572,529.50","121,834,418.00","121,592,715.00","115,027,569.00"


In [57]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"857,651,837.00"
2004,"939,332,777.50"
2005,"973,180,872.50"
2006,"926,806,118.50"
2007,"995,701,834.00"
2008,"1,007,961,369.50"
2009,"927,852,836.50"
2010,"849,411,941.50"
2011,"998,363,305.00"


In [58]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,055,133,014.50"
2,"798,468,217.00"
3,"868,419,796.00"
4,"686,397,624.00"
5,"668,475,966.00"
6,"543,744,320.50"
7,"570,650,264.50"
8,"629,370,769.50"
9,"785,247,040.50"


In [59]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False)

windfarm
year month               
2003 1     109,609,389.50
     2      57,162,630.00
     3      65,764,028.50
     4      89,456,376.50
     5      59,373,813.00
...                   ...
2012 8      52,303,476.50
     9      92,845,808.50
     10     92,746,242.50
     11     91,565,259.00
     12    107,745,710.50

[120 rows x 1 columns]

<a id="net-present-value"></a>
## Net Present Value

Here, we will go through the various input definitions to get the project NPV

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`discount_rate`: The rate of return that could be earned on alternative investments, by default 0.025. 

`offtake_price`: Price of energy, per MWh, by default 80.

In [60]:
metrics.npv("project")

,NPV
0,"199,542,153.22"


In [61]:
metrics.opex("annual")

,OpEx
year,
2003,"9,855,094.81"
2004,"11,582,131.37"
2005,"12,043,947.80"
2006,"17,184,945.50"
2007,"14,308,828.64"
2008,"14,964,709.29"
2009,"13,051,393.09"
2010,"11,301,060.97"
2011,"16,463,100.53"


In [62]:
metrics.opex("monthly")

,OpEx
month,
1,"10,972,230.89"
2,"11,154,966.54"
3,"13,695,271.15"
4,"13,767,942.46"
5,"17,864,156.56"
6,"8,939,017.24"
7,"11,609,726.89"
8,"13,074,295.45"
9,"12,192,580.28"


In [63]:
metrics.opex("month-year")

OpEx
year month             
2003 1       367,339.01
     2       283,703.88
     3       356,120.26
     4       343,869.49
     5       372,057.76
...                 ...
2012 8     2,548,038.66
     9     4,136,986.67
     10    1,500,776.51
     11      369,082.15
     12      745,643.69

[120 rows x 1 columns]

<a id="pysam-powered-results"></a>
## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [64]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [65]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [66]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [67]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [68]:
try:
    metrics.pysam_all_outputs()
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_all_outputs()' cannot be calculated!
